# Загрузка библиотек.

In [1]:
import pandas as pd, numpy as np
from math import sqrt
from sklearn.metrics.pairwise import euclidean_distances

# Загрузка датасэтов.

## Значения датасэтов:
- df_mtd - датасэт, содержащий все тренировочные значения.
- df_cptd - датасэт, содержащий тренировочные значения заказчик-поставщик-количество взаимодействий.
- df_atd - датасэт, содержащий аггрегированные тренировочные значения о каждом поставщике.

In [2]:
df_mtd = pd.read_csv('merged_training_data.csv', delimiter = ';')
df_cptd = pd.read_csv('customer_participant_training_data.csv', delimiter = ';')
df_atd = pd.read_csv('aggregated_training_data.csv', delimiter = ';')

/tmp/ipykernel_3687/2292234994.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mtd = pd.read_csv('merged_training_data.csv', delimiter = ';')


**Немного изменим структуру датасэта df_atd.**

In [3]:
df_atd.set_index(keys = 'Participant', inplace = True)
df_atd = df_atd[['Fz', 'Region code', 'Etp', 'Participated in small business', 
                 'Month', 'Lot price category', 'Code', 'Times participated', 'Winning percentage']]

# Обработка входных значений.

**Выбор случайного лота.**

In [4]:
lot = df_mtd.sample(1)
lot

,fz,lot,region_code,etp,forsmallbiz,lot_price,customer,okpd2_code,participant,winner,Month,Lot price category
5268592,1,8e0d2cc2cbdba85bf59318bbaf088b5f,55,14.0,1,300000.0,4cc15a93de89f8c18227b949da7224d1,81.2,aeb48f3567d700415365343be127e9b2,1.0,11,3.0


**Отображение полной информации по данному лоту.**

In [5]:
df_lot = df_mtd[df_mtd['lot'] == lot.iloc[0, 1]]
df_lot

,fz,lot,region_code,etp,forsmallbiz,lot_price,customer,okpd2_code,participant,winner,Month,Lot price category
5268592,1,8e0d2cc2cbdba85bf59318bbaf088b5f,55,14.0,1,300000.0,4cc15a93de89f8c18227b949da7224d1,81.2,aeb48f3567d700415365343be127e9b2,1.0,11,3.0


**Добавление всех поставщиков, принимающих участие в этом тендере, в массив действительных поставщиков.**

In [6]:
actual_participants = df_lot['participant'].to_numpy()
actual_participants

array(['aeb48f3567d700415365343be127e9b2'], dtype=object)

# Рекомендательная система.

**Создание массива для рекомендованных поставщиков.**

In [7]:
recommended_participants = np.array([])

## 1-ый этап рекомендательной системы: рекомендации на основе истории взаимоотношений.

**Отображение всех поставщиков, с кем работал заказчик выбранного лота.**

In [8]:
df_cptd_lot = df_cptd[df_cptd['customer'] == lot.iloc[0, 6]]
df_cptd_lot

,customer,participant,count
517163,4cc15a93de89f8c18227b949da7224d1,460a769ad49500ef7c9f73dbccbd74ac,4
517177,4cc15a93de89f8c18227b949da7224d1,661628b25d61aad9a63d3b3073081dd4,4
913836,4cc15a93de89f8c18227b949da7224d1,ea25cae725e202f917704413912f0907,3
913878,4cc15a93de89f8c18227b949da7224d1,d92f29cdeb40d1d35c26b16a1235c356,3
2428605,4cc15a93de89f8c18227b949da7224d1,2399e028295c55ecabc8ba9ea76364f7,2
...,...,...,...
2468253,4cc15a93de89f8c18227b949da7224d1,decd49372a8c396ae8f2aaff27779eda,1
2468254,4cc15a93de89f8c18227b949da7224d1,f73bd51091395f6556cc1786bf8f41ce,1
2468255,4cc15a93de89f8c18227b949da7224d1,f99becf0840c6fdd400eaf10b03d337e,1
2468256,4cc15a93de89f8c18227b949da7224d1,fee37c2325896ebaa08ab5da160b95ef,1


**Процедура, выбирающая первых трёх (в порядке убывания количества взаимодействий) поставщиков, которые хотя бы раз играли в том же коде ОКПД2.**

In [9]:
for row in range(len(df_cptd_lot)):
    if len(df_mtd[(df_mtd['participant'] == df_cptd_lot.iloc[row, 1])
           & (df_mtd['okpd2_code'] == lot.iloc[0, 7])]) != 0:
        recommended_participants = np.append(recommended_participants, df_cptd_lot.iloc[row, 1])
        if len(recommended_participants) == 3: 
            break

In [10]:
recommended_participants

array(['661628b25d61aad9a63d3b3073081dd4',
       '41ca6b5180f5775519a9aebe52e1373a',
       '913cc257372caf97987862cf90e5ddd1'], dtype='<U32')

## 2-ой этап рекомендательной системы: рекомендации на основе похожести поставщиков и лота.

**Создаём датафрейм, содержащий характеристики лота.**

In [11]:
df_lot_features = lot.copy()
df_lot_features.drop(['lot', 'lot_price', 'customer', 'okpd2_code', 
                      'participant', 'winner'], axis = 1, inplace = True)
df_lot_features

,fz,region_code,etp,forsmallbiz,Month,Lot price category
5268592,1,55,14.0,1,11,3.0


## Используем алгоритм euclidean distances, чтобы найти максимально поставщиков с максимально похожими характеристиками.

**Редактируем датасэт df_atd для дальнейшего использования.**

In [12]:
df_atd_features = df_atd[(df_atd['Code'] == df_lot.iloc[0, 7])
                         & (df_atd['Fz'] == df_lot.iloc[0, 0])].copy()
if df_lot.iloc[0, 4] == 1:
    df_atd_features = df_atd_features[df_atd_features['Participated in small business'] == 1]
df_atd_features.drop('Code', axis = 1, inplace = True)

df_atd_features

,Fz,Region code,Etp,Participated in small business,Month,Lot price category,Times participated,Winning percentage
Participant,,,,,,,,
1bfd55aba23d232c5e46b9cd6e651365,1,77,20.0,1,9,5.0,3475,0.0
7857f1a348d9bc0db2b7c2c0b1080bf7,1,77,20.0,1,3,5.0,2712,0.0
e30ea198af1d1e0590902d2404be3262,1,77,20.0,1,11,5.0,1489,3.0
756e3b3c107ad5112f7582b6e4807974,1,77,20.0,1,12,5.0,1158,0.0
146606a064091f61e62df032103a7c20,1,78,18.0,1,12,1.0,1011,35.0
...,...,...,...,...,...,...,...,...
fdd9f5b7edc40317e9068e56ec6434f6,1,77,20.0,1,4,2.0,1,0.0
fdb1961255f1bbe978d155dd61ed8a53,1,65,7.0,1,11,2.0,1,100.0
fdf198fc0d272e6531e94e374720f59e,1,21,20.0,1,12,4.0,1,0.0


In [13]:
df_atd_features['Etp'].fillna(method = 'pad', inplace = True)
df_atd_features['Lot price category'].fillna(method = 'pad', inplace = True)

**Присваиваем значения и находим наименьшое Евклидово расстояние. x - характеристики лота, y - характеристики поставщика.**

In [15]:
x = df_lot_features

while len(recommended_participants) != 5:
    ed_reference = euclidean_distances(x, df_atd_features.iloc[0, :6].to_frame().transpose())
    
    for row in range(1, len(df_atd_features)):
        y_reference = df_atd_features.iloc[row, :]
        ed = euclidean_distances(x, y_reference.iloc[:6].to_frame().transpose())
        if ed < ed_reference:
            ed_reference = ed
            y = y_reference

    recommended_participants = np.append(recommended_participants, y.name)
    df_atd_features.drop(labels = y.name, axis = 0, inplace = True)

# Оценка рекомендательной системы.

**Список рекомендованных поставщиков.**

In [16]:
recommended_participants

array(['661628b25d61aad9a63d3b3073081dd4',
       '41ca6b5180f5775519a9aebe52e1373a',
       '913cc257372caf97987862cf90e5ddd1',
       'c642c50e769f19298e262561339b1f42',
       'd550498448e48b4ffb2895f2c128905d'], dtype='<U32')

**Список действительных поставщиков.**

In [17]:
actual_participants

array(['aeb48f3567d700415365343be127e9b2'], dtype=object)

**Считаем процент правильных прогнозов.**

In [18]:
isin = np.isin(actual_participants, recommended_participants)
np.count_nonzero(isin == True) / len(isin)

0.0